In [6]:
import time
import functools
import random
from pathlib import Path
import numpy as np
import copy
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler, swa_utils
import torchvision
from torchvision.datasets import ImageFolder, DatasetFolder
from torchvision import transforms
from diskcache import Cache

In [7]:

cache = Cache('.cache/diskcache')
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'data/face_age'


@cache.memoize()
def imgpath_to_normalized_tensor(imgpath):
        # makes a tensor, scales range to 0-1 and normalizes to same as imagenet
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
        img = normalize(transforms.PILToTensor()(Image.open(imgpath)).float()/255)
        return img


class Ageset(Dataset):
    def __init__(self, path, transforms = None, valid=False, split_pct = 0.3):
        self.image_paths = list(Path(path).rglob("*.png"))
        random.seed(42)
        random.shuffle(self.image_paths)
        split_point = int(len(self)*0.3)
        if valid:
            self.image_paths = self.image_paths[:split_point]
            print('len validation dataset', len(self.image_paths))
        else:
            self.image_paths = self.image_paths[split_point:]
            print('len train dataset', len(self.image_paths))
    def __len__(self):
        return len(self.image_paths)

    # @functools.lru_cache(maxsize=None)
    @classmethod # somehow this is needed for diskcache to work properly. Or define the function outside of the class
    @cache.memoize()
    def imgpath_to_normalized_tensor(cls,imgpath):
            # makes a tensor, scales range to 0-1 and normalizes to same as imagenet
            normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
            img = normalize(transforms.PILToTensor()(Image.open(imgpath)).float()/255)
            return img
            
    def __getitem__(self,i):
        if isinstance(i, slice):
            return [self[n] for n,_ in enumerate(self.image_paths[i])]
        
        return (self.imgpath_to_normalized_tensor(self.image_paths[i]),
                int(self.image_paths[i].parent.name))

In [8]:
def construct_tensor_dataset(path, max_len=None):
    # this took 20 GB RAM max
    image_paths = list(Path(path).rglob("*.png"))
    random.shuffle(image_paths)
    if max_len:
        image_paths = image_paths[:max_len]
    
    xs = torch.empty(len(image_paths), 3,200,200)
    print('empty created')
    for i, loc in enumerate(image_paths):
        if i%1000==0: print(i)
        xs[i] = imgpath_to_normalized_tensor(loc)
    torch.save(xs,'data/input/xs')

    print('done')

    ys = torch.stack([torch.Tensor([int(Path(loc).parent.name)]) for loc in image_paths])
    torch.save(ys,'data/input/ys')
    print('done')
    return None
# construct_tensor_dataset(PATH)

In [9]:
class AgeTensorDataset(TensorDataset):
    def __init__(self, xs, ys, valid=False, split_pct=0.3):
        length = len(xs)
        split = int(xs.shape[0]*split_pct)
        if valid:
            super().__init__(xs[:split],ys[:split])
        else:
            super().__init__(xs[split:],ys[split:])

    def __getitem__(self,x):
        return super().__getitem__(x)

In [10]:
class AgeResnet(nn.Module):
    def __init__(self, size='18', feat_extract=False):
        super().__init__()
        resnet = 'torchvision.models.resnet'+size+'(pretrained=True)'
        resnet = eval(resnet)
        modules=list(resnet.children())[:-1]
        self.resnet =nn.Sequential(*modules)

        if feat_extract:
            # with feature extraction we only train the linear layer and keep the resnet parameters fixed 
            for m in self.modules():
                m.requires_grad_(False)

        self.fc = nn.Linear(in_features=512, out_features=1, bias=True)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self,x):
        out = self.resnet(x)
        x = torch.flatten(out, 1)
        return self.fc(x)

In [11]:
def determine_size(dataset):
    num_items = len(dataset)
    img_dimensions = list(dataset[0][0].shape)
    bytes_per_fp32 = 4
    bytes_per_gb = 1024**3
    size_in_gb = num_items * int(np.product(img_dimensions)) * bytes_per_fp32 / bytes_per_gb
    print('items in dataset', num_items, 'img_dimensions', img_dimensions, 'size of ds in memory in gb:', size_in_gb)

In [20]:
def train():
    best_loss = 1000000000
    best_model = None
    not_improve_count = 0
    loss = {'train':[], 'val':[]}

    for epoch in range(NUM_EPOCH):
        print(f'Starting epoch {epoch}')
        start_time = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            total_loss = 0
            for data in dls[phase]:
                x, y = data[0].to(DEVICE), data[1].to(DEVICE)
                with torch.set_grad_enabled(phase == 'train'):
                    pred = model(x)
                    loss = loss_fn(y, pred)
                    total_loss += loss * len(y)
                    if phase == 'train':
                        loss.backward()
                        opt.step()
                        opt.zero_grad()
                        if SWA_ENABLED and epoch > SWA_START:
                            swa_model.update_parameters(model)
                            swa_sched.step()
                        elif SCHED_ENABLED:
                            sched.step(loss)
                            writer.add_scalar('lr/scheduler', sched.get_last_lr()[0], epoch)
                        writer.add_scalar('lr/optparamgroup0', opt.param_groups[0]['lr'], epoch)
                writer.add_scalar('batchloss/'+phase, loss, epoch)
            
            writer.add_scalar('loss/'+phase, total_loss/len(dls[phase].dataset), epoch)
    
        if total_loss < best_loss:
            best_loss = total_loss
            best_model = copy.deepcopy(model.state_dict())
            not_improve_count = 0
        else:
            not_improve_count += 1
            if not_improve_count > 4:
                print('early stopping!')
                break

        for x,y in val_set[:10]:
            x = x.to(DEVICE)
            print (model(x[None]),y)
            
        print(f"loss after epoch {epoch} : {total_loss / len(dls['val'].dataset)}")
        writer.add_scalar('time', (time.time()-start_time)/60, epoch)


    if SWA_ENABLED:
        swa_model.to('cpu')
        swa_utils.update_bn(train_dl, swa_model)
        swa_model.to(DEVICE)
        total_loss_train = 0
        total_loss_val = 0
        with torch.no_grad():
            for data in train_dl:
                    x, y = data[0].to(DEVICE), data[1].to(DEVICE)
                    total_loss_train += loss_fn(y, model(x)) * len(y)
            writer.add_scalar('loss/train', total_loss_train/len(train_set), epoch+1)

            for data in val_dl:
                    x, y = data[0].to(DEVICE), data[1].to(DEVICE)
                    total_loss_val += loss_fn(y, model(x)) * len(y)
            writer.add_scalar('loss/val', total_loss_train/len(val_set), epoch+1)
        swa_model.avg_fn=None
        torch.save(swa_model,'data/output/'+'swa_model'+str(total_loss_val/len(val_set)))
    torch.save(best_model,'data/output/'+'model'+str(best_loss/len(val_set)))
    writer.flush()
    writer.close()

In [21]:
# adam works best with lr of 0.001 (tested 0.1 and 0.01)
# adam without scheduler works best
# first it took 6 minutes to load all the datasets. With lru cache it was immediate (6GB memory use). With disk cache it took about 1-2 minutes. Great result
# feature extraction led to MSE of 12 after 40 epochs. Didnt really work. Maybe unfreeze more
# adam was outperformed in feature extraction, but for finetuning it worked better
# larger bs converges better and runs sligthly faster (about 10%) [512, 256,64,8]

import adabound

def mae_loss(y, pred):
    return (torch.abs(y-pred.T)).mean()
loss_fn = mae_loss

NUM_EPOCH = 50
SWA_START = 40
LR = 0.001
BATCH_SIZE = 512
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
SWA_ENABLED = False
SCHED_ENABLED = False

torch.cuda.empty_cache()
# xs = torch.load('data/input/xs').to(DEVICE)
# ys = torch.load('data/input/ys').to(DEVICE)

# train_set = AgeTensorDataset(xs,ys, valid=False)
train_set = Ageset("data/face_age")[:4000]
determine_size(train_set)
train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

# val_set = AgeTensorDataset(xs,ys, valid=True)
val_set = Ageset("data/face_age", valid=True)[:1000]
determine_size(val_set) 
val_dl = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
dls = {'train': train_dl, 'val': val_dl}

feat={True:'feat_ext', False:'finetune'}
opts = {0:'adam',1:'adabound'}

for i in [False]:
    for j in range(1):
        for BATCH_SIZE in [256]:
            for res in [18,50,152]:

                torch.cuda.empty_cache()



                writer = SummaryWriter(comment=f'{feat[i]} opt {opts[j]} epoch {NUM_EPOCH} SWA_START {SWA_START} LR BATCH_SIZE {LR}')
                model = AgeResnet(size=str(res), feat_extract=i)
                model = model.to(DEVICE)

                if j ==0:
                    opt = torch.optim.Adam(model.parameters(), LR)
                if j ==1:
                    opt = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)


                if SCHED_ENABLED:
                    # if i ==0:
                    #     sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)
                    #     sched.get_last_lr = lambda: [1]
                    # if i == 1:
                    #     sched = torch.optim.lr_scheduler.OneCycleLR(opt, LR, steps_per_epoch=len(train_dl), epochs=NUM_EPOCH)
                    # if i == 2:
                    sched = torch.optim.lr_scheduler.MultiplicativeLR(opt, lr_lambda=lambda x: 1)
                if SWA_ENABLED:
                    swa_model = swa_utils.AveragedModel(model)
                    swa_sched = swa_utils.SWALR(opt, swa_lr = 0.0005)
                train()

len train dataset 13690
items in dataset 4000 img_dimensions [3, 200, 200] size of ds in memory in gb: 1.7881393432617188
len validation dataset 5866
items in dataset 1000 img_dimensions [3, 200, 200] size of ds in memory in gb: 0.4470348358154297
Starting epoch 0
tensor([[20.5073]], device='cuda:0', grad_fn=<AddmmBackward>) 32
tensor([[-51.7517]], device='cuda:0', grad_fn=<AddmmBackward>) 1
tensor([[32.9655]], device='cuda:0', grad_fn=<AddmmBackward>) 50
tensor([[23.4655]], device='cuda:0', grad_fn=<AddmmBackward>) 85
tensor([[26.5001]], device='cuda:0', grad_fn=<AddmmBackward>) 50
tensor([[15.9190]], device='cuda:0', grad_fn=<AddmmBackward>) 19
tensor([[-41.8627]], device='cuda:0', grad_fn=<AddmmBackward>) 1
tensor([[-7.7527]], device='cuda:0', grad_fn=<AddmmBackward>) 2
tensor([[4.0758]], device='cuda:0', grad_fn=<AddmmBackward>) 8
tensor([[16.2037]], device='cuda:0', grad_fn=<AddmmBackward>) 18
loss after epoch 0 : 25.10700035095215
Starting epoch 1
tensor([[29.1800]], device='cuda

NameError: name 'swa_model' is not defined

In [31]:
model = AgeResnet()
model.load_state_dict(torch.load("data/output/modeltensor(4.3645, device='cuda:0')"))
# for i in test_set[:10]:
#     preds = i[1],model(i[0][None].to(DEVICE)).item()
#     print(f'target {preds[0]}, predicted {preds[1]}')
#     loss = abs(preds[0]-preds[1])
#     # print('loss',loss)
#     running += loss
# print(running, running/2)


<All keys matched successfully>

In [48]:
val_set = Ageset("data/face_age", valid=True)[:1000]
determine_size(val_set) 
val_dl = DataLoader(val_set, batch_size=8, shuffle=True)
x,y = next(iter(val_dl))
x.to(DEVICE)
model.eval()
print (model(x),y)
mae_loss(y, model(x))

len validation dataset 5866
items in dataset 1000 img_dimensions [3, 200, 200] size of ds in memory in gb: 0.4470348358154297
tensor([[  5.4504],
        [ 64.6927],
        [ 40.4678],
        [  2.7160],
        [ 83.1447],
        [ 43.6335],
        [115.4910],
        [108.6111]], grad_fn=<AddmmBackward>) tensor([ 5, 53, 15,  1, 60, 30, 80, 85])


tensor(16.9009, grad_fn=<MeanBackward0>)

In [61]:
for x,y in val_set[:10]:
    model.eval()
    x.to(DEVICE)
    print(model(x[None]),y)

tensor([[39.5846]], grad_fn=<AddmmBackward>) 32
tensor([[-0.8336]], grad_fn=<AddmmBackward>) 1
tensor([[124.0162]], grad_fn=<AddmmBackward>) 50
tensor([[168.4021]], grad_fn=<AddmmBackward>) 85
tensor([[105.6732]], grad_fn=<AddmmBackward>) 50
tensor([[32.9609]], grad_fn=<AddmmBackward>) 19
tensor([[-0.9324]], grad_fn=<AddmmBackward>) 1
tensor([[0.1149]], grad_fn=<AddmmBackward>) 2
tensor([[4.1401]], grad_fn=<AddmmBackward>) 8
tensor([[21.9747]], grad_fn=<AddmmBackward>) 18


In [50]:
model(val_set[1][0][None])

tensor([[3.3705]], grad_fn=<AddmmBackward>)

In [ ]:
#launch tensorboard
python -m tensorboard.main --logdir=runs --host=0.0.0.0 --port=6006

In [ ]:
tensorboard dev upload --logdir runs \
--name "My latest experiment" \ # optional
--description "Simple comparison of several hyperparameters" # optional

In [18]:
torch.save(model.state_dict(),'data/output/bestmodel.model')

In [17]:
a = torch.Tensor([2])
b = a.to(DEVICE)
a.device, b.device

(device(type='cpu'), device(type='cuda', index=0))

In [2]:
import torch

In [8]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [15]:
torch.cuda.empty_cache()